In [73]:
# DEFINING REQUIRED PACKAGES ---------------------------------------
import matplotlib.pyplot as plt #import matplotlib to create plots
import scipy.optimize as opt; #allows for curve fitting
from scipy.odr import *
import pandas as pd #importing pandas to use dataframes
import numpy as np
from scipy.constants import g, pi, e
plt.style.reload_library()
plt.style.use(['science', 'grid'])
from scipy.odr import ODR, Model, Data, RealData
from scipy.optimize import curve_fit
import ipywidgets as widgets
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [74]:
# DEFINING CONSTANTS
M, M_delta = 2.04, 0.05
oil, oil_delta = 872, 0
viscosity, viscosity_delta = 1.85e-5, 0
air, air_delta = 1.2, 0
d, d_delta = 5.91e-3, 0.05*1e-3
A, A_delta = 0.07776e-6, 0
V, V_delta = 500, 1

# FUNCTIONS
def v(param): #[y,t,M]
    return 0.001*param[0]/(param[1]*param[2])

def r(param): #[viscosity,v_g,oil,air]
    return np.sqrt((9*param[0]*param[1])/(2*(param[2]-param[3])*g))

def q(param): #[viscosity, d, vg, vu, r, V]
    return 6*pi*param[0]*param[1]*(param[2]+param[3])*param[4]/param[5]

def qc(param): #[q,A,r]
    return param[0]/(1+param[1]/param[2])**(3/2)

def N(param): #[qc]
    return param[0]/e

In [75]:
# READING THE DATA FROM GOOGLE SHEETS ---------------------------------------
googleSheetId = '1AAvU0Xu4zl96DeL4DVbLMTxFEpD9nzQF2a73egXsyKc'
workSheetName = 'PreLab'
URL = 'https://docs.google.com/spreadsheets/d/{0}/gviz/tq?tqx=out:csv&sheet={1}'.format(
    googleSheetId, workSheetName)
df = pd.read_csv(URL)
df = df.iloc[: , :15]
df

,y1 (mm),Δy1 (mm),Time1 (s),Δt1 (s),v_g (m/s),y2 (mm),Δy2 (mm),Time2 (s),Δt2 (s),v_u (m/s),V (V),r,q,qc,N
0,1,0.1,15.92,0.01,0.00003,1,0.1,6.82,0.01,0.00007,500,5.500000e-07,2.320000e-19,1.900000e-19,1.19


In [76]:
# DEFINING A FUNCTION THAT RETURNS THE UNCERTAINTY USING VARIATIONAL METHOD (Each function must receive a parameters array)
def uncertainty_var(values, deltas, function):
    values, deltas = np.array(values), np.array(deltas)
    delta_matrix = deltas*np.identity(values.shape[0])
    sum = 0
    for delta in delta_matrix:
        sum += (0.5*(abs(function(values + delta) - function(values)) + abs(function(values-delta) - function(values))))**2
    return np.sqrt(sum)

In [77]:
df['$\Delta v_g$'] = df[['y1 (mm)','Δy1 (mm)','Time1 (s)','Δt1 (s)']].apply(lambda x: uncertainty_var([x[0],x[2],M],[x[1],x[3],M_delta],v),axis=1)
df['$\Delta v_u$'] = df[['y2 (mm)','Δy2 (mm)','Time2 (s)','Δt2 (s)']].apply(lambda x: uncertainty_var([x[0],x[2],M],[x[1],x[3],M_delta],v),axis=1)
df['$\Delta r$'] = df[['v_g (m/s)', '$\Delta v_g$']].apply(lambda x: uncertainty_var([viscosity,x[0],oil,air],[viscosity_delta,x[1],oil_delta,air_delta],r),axis=1)
df['$\Delta q$'] = df[['v_g (m/s)', 'v_u (m/s)', 'r', '$\Delta v_g$', '$\Delta v_u$', '$\Delta r$']].apply(lambda x: uncertainty_var([viscosity, d, x[0], x[1], x[2], V],[viscosity_delta, d_delta, x[3],x[4],x[5],V_delta],q),axis=1)
df['$\Delta qc$'] = df[['q', 'r', '$\Delta q$', '$\Delta r$']].apply(lambda x: uncertainty_var([x[0],A,x[1]],[x[2],A_delta,x[3]],qc),axis=1)
df['$\Delta N$'] = df[['qc', '$\Delta qc$']].apply(lambda x: uncertainty_var([x[0]],[x[1]],N),axis=1)

In [78]:
df

,y1 (mm),Δy1 (mm),Time1 (s),Δt1 (s),v_g (m/s),y2 (mm),Δy2 (mm),Time2 (s),Δt2 (s),v_u (m/s),...,r,q,qc,N,$\Delta v_g$,$\Delta v_u$,$\Delta r$,$\Delta q$,$\Delta qc$,$\Delta N$
0,1,0.1,15.92,0.01,0.00003,1,0.1,6.82,0.01,0.00007,...,5.500000e-07,2.320000e-19,1.900000e-19,1.19,0.000003,0.000007,2.861596e-08,2.182198e-20,1.799029e-20,0.112287
